In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class DualSourcing():
    #config=dictionary of {"regular_leadtime":XXX,...}
    #y is the discount factor
    #demand=a list of len(episode_len*episode_rep), and initializes to poisson distribution
    def __init__(self, config,episode_len=1000,episode_rep=50,demand=None):
        self.rl = config['regular_leadtime']
        self.el = config['express_leadtime']
        self.rc = config['regular_cost']
        self.ec = config['express_cost']
        self.sc = config['store_cost']
        self.bc = config['back_cost']
        self.max_order=config['max_order']
        self.max_inventory=config['max_inventory']
        self.y=config['y']
        self.starting_state=config['starting_state']
        self.episode_len=episode_len
        self.episode_rep=episode_rep
        self.total_episode_len=self.episode_len*self.episode_rep
        if demand==None:
            #simulate an iid demand with constant+poisson random variable
            self.poisson_lambda=4
            self.fixed_demand=2
            demand=np.random.poisson(self.poisson_lambda,self.total_episode_len)
            demand=[d+self.fixed_demand for d in demand]
        if len(demand)!=self.total_episode_len:
            raise ValueError("Length of Demand {} must match episode length {}".format(len(demand),self.total_episode_len))
        self.demand=demand
#         state=[cur_inventory,[reg_order_by_rl,reg_order_by_rl-1,reg_order_by_rl-2...,reg_order_just_made],
#                [exp_order_by_el,exp_order_by_el-1...exp_order_just_made]]
        self.state = self.starting_state
        self.inventory_overflow=0
        self.inventory_underflow=0
        self.inventory_history=[]
        self.reward_history=[]
        self.demand_history=[]
        self.reg_order_history=[]
        self.exp_order_history=[]
        self.total_cost=0
        self.time_stamp=0
    # Auxilary function 
    #update inventory and pipeline vectors
    def update_inventory(self,demand,r_order,e_order):
        cur_inventory=self.state[0]+self.state[1][0]+self.state[2][0]-demand
        if cur_inventory>self.max_inventory:
            self.inventory_overflow=cur_inventory-self.max_inventory
            cur_inventory=self.max_inventory
        else:
            self.inventory_overflow=0
        if cur_inventory<-self.max_inventory:
            self.inventory_underflow=-self.max_inventory-cur_inventory
            cur_inventory=-self.max_inventory
        else:
            self.inventory_underflow=0
        self.state[0]=cur_inventory
        self.state[1]=self.state[1][1:]+[r_order]
        self.state[2]=self.state[2][1:]+[e_order]
        return self.state[0]
    #compute reward, the negative of cost
    #we modified the reward term to penalize 
    #i)exceeding max stock or dropping below min stock
    #ii)ordering much when stock is already high; constantly pushing the stock up
    #iii)ordering little when stock is negative
    #in other words, we would like to encourage the agent to push inventory towards 0, but never falls below 0 
    def calcuate_reward(self,r_order,e_order):
        cost=(self.rc*r_order+self.ec*e_order+self.sc*max(self.state[0],0)+self.bc*max(-self.state[0],0))
        self.total_cost+=cost
        stocking_penality=0
        insufficiency_penalty=0
        overflow_penalty=0
        underflow_penalty=0
        past_demand=self.demand_history[self.time_stamp-1]
        if self.time_stamp>1 and self.state[0]>(self.el)*past_demand:
            stocking_penality=2.4*self.sc*(e_order)
        if self.state[0]+self.state[1][0]+self.state[2][0]<0:
            insufficiency_penalty=5*self.bc*(-self.state[0]-r_order-e_order)
        if self.inventory_overflow>0:
            overflow_penalty=8.2*self.sc*(self.inventory_overflow+r_order+e_order)
        if self.inventory_underflow>0:
            underflow_penalty=6.4*self.bc*(self.inventory_underflow-r_order-e_order)
        return -(cost+stocking_penality+insufficiency_penalty+overflow_penalty+underflow_penalty)
    def reset(self):
        self.state = np.asarray(self.starting_state)
        self.inventory_history=[]
        self.reward_history=[]
        self.demand_history=[]
        self.reg_order_history=[]
        self.exp_order_history=[]
        self.time_stamp=0
        return self.state 
    def plot(self,time_start,time_end):
        fig=plt.figure(figsize=[12,10])
        ax_inv=fig.add_subplot(3,1,1)
        ax_inv.set_xlabel("TimeStamp")
        ax_inv.set_ylabel("Inventory Level")
        ax_demand=fig.add_subplot(3,1,2)
        ax_demand.set_xlabel("TimeStamp")
        ax_demand.set_ylabel("Demand Level")
        ax_orders=fig.add_subplot(3,1,3)
        ax_orders.set_xlabel("TimeStamp")
        ax_orders.set_ylabel("Order Quantity")
        ax_inv.plot(list(range(time_start%self.episode_len,time_end%self.episode_len+1)),self.inventory_history[time_start:time_end+1])
        ax_demand.plot(list(range(time_start%self.episode_len,time_end%self.episode_len+1)),self.demand_history[time_start:time_end+1])
        ax_orders.plot(list(range(time_start%self.episode_len,time_end%self.episode_len+1)),self.reg_order_history[time_start:time_end+1],label="Regular Orders")
        ax_orders.plot(list(range(time_start%self.episode_len,time_end%self.episode_len+1)),self.exp_order_history[time_start:time_end+1],label="Express Orders")
        ax_orders.legend()
        plt.show()
    #this function allows users to play the game
    def play(self):
        while(self.time_stamp<self.episode_len):
            cur_demand=self.demand[self.time_stamp]
            self.time_stamp+=1
            print("Current Demand is "+str(cur_demand))
            print("Your current inventory level is "+str(self.state[0]))
            print("Your Pipeline vector from regular supplier is"+str(self.state[1]))
            print("Your Pipeline vector from express supplier is"+str(self.state[2]))
            print("Input your regular and express orders,separated by comma")
            str_inp=str(input()).split(",")
            order_reg=int(str_inp[0])
            order_exp=int(str_inp[1])
            self.inventory_history.append(self.update_inventory(cur_demand,order_reg,order_exp))
            self.demand_history.append(cur_demand)
            self.reward_history.append(self.calcuate_reward(order_reg,order_exp))
            self.reg_order_history.append(order_reg)
            self.exp_order_history.append(order_exp)
            fig=plt.figure(figsize=[12,10])
            ax_inv=fig.add_subplot(2,2,1)
            ax_inv.set_xlabel("TimeStamp")
            ax_inv.set_ylabel("Inventory Level")
            ax_reward=fig.add_subplot(2,2,2)
            ax_reward.set_xlabel("TimeStamp")
            ax_reward.set_ylabel("Average Reward Level")
            ax_demand=fig.add_subplot(2,2,3)
            ax_demand.set_xlabel("TimeStamp")
            ax_demand.set_ylabel("Demand Level")
            ax_orders=fig.add_subplot(2,2,4)
            ax_orders.set_xlabel("TimeStamp")
            ax_orders.set_ylabel("Order Quantity")
            ax_inv.plot(list(range(self.time_stamp)),self.inventory_history)
            ax_reward.plot(list(range(self.time_stamp)),[val/(index+1) for index,val in enumerate(self.reward_history)])
            ax_demand.plot(list(range(self.time_stamp)),self.demand_history)
            ax_orders.plot(list(range(self.time_stamp)),self.reg_order_history,label="Regular Orders")
            ax_orders.plot(list(range(self.time_stamp)),self.exp_order_history,label="Express Orders")
            ax_orders.legend()
            plt.show()
    def step(self,reg_order,exp_order):
            if self.time_stamp==self.total_episode_len:
                raise ValueError("Time stamp {} must be less or equal to length of episode".format(self.time_stamp+1,self.episode_len))
            cur_demand=self.demand[self.time_stamp]
            self.time_stamp+=1
            self.demand_history.append(cur_demand)
            self.inventory_history.append(self.update_inventory(cur_demand,reg_order,exp_order))
            self.reward=self.calcuate_reward(reg_order,exp_order)
            self.reward_history.append(self.reward)
            self.reg_order_history.append(reg_order)
            self.exp_order_history.append(exp_order)
            if self.time_stamp==self.total_episode_len:
                fig=plt.figure(figsize=[12,10])
                ax_inv=fig.add_subplot(2,2,1)
                ax_inv.set_xlabel("TimeStamp")
                ax_inv.set_ylabel("Inventory Level(last episode)")
                ax_reward=fig.add_subplot(2,2,2)
                ax_reward.set_xlabel("Episode Number")
                ax_reward.set_ylabel("Average Reward Level")
                ax_demand=fig.add_subplot(2,2,3)
                ax_demand.set_xlabel("TimeStamp")
                ax_demand.set_ylabel("Demand Level(last episode)")
                ax_orders=fig.add_subplot(2,2,4)
                ax_orders.set_xlabel("TimeStamp")
                ax_orders.set_ylabel("Order Quantity(last episode)")
                ax_inv.plot(list(range(self.time_stamp-self.episode_len,self.time_stamp)),self.inventory_history[self.time_stamp-self.episode_len:])
                reward_matrix=np.reshape(self.reward_history,(self.episode_rep,self.episode_len))
                avg_rewards=np.mean(reward_matrix,axis=1)
                ax_reward.plot(avg_rewards)
                ax_demand.plot(list(range(self.time_stamp-self.episode_len,self.time_stamp)),self.demand_history[self.time_stamp-self.episode_len:])
                ax_orders.plot(list(range(self.time_stamp-self.episode_len,self.time_stamp)),self.reg_order_history[self.time_stamp-self.episode_len:],label="Regular Orders")
                ax_orders.plot(list(range(self.time_stamp-self.episode_len,self.time_stamp)),self.exp_order_history[self.time_stamp-self.episode_len:],label="Express Orders")
                ax_orders.legend()
                plt.show()
                print("Info on Last Episode:")
                print("Avg Inventory Stock")
                avg_inventory=sum(self.inventory_history[self.time_stamp-self.episode_len:])/self.episode_len
                print(avg_inventory)
                print("Variance of Inventory Stock")
                var_inventory = sum((x - avg_inventory) ** 2 for x in self.inventory_history[self.time_stamp-self.episode_len:]) /self.episode_len
                print(var_inventory)
                print("Avg Inventory Holding(多出来的部分)")
                total_overflow=sum(max(0,self.inventory_history[i]) for i in range(self.time_stamp-self.episode_len,self.time_stamp))
                print(total_overflow/self.episode_len)
                print("Avg Inventory Backloss")
                total_backloss=sum(-min(0,self.inventory_history[i]) for i in range(self.time_stamp-self.episode_len,self.time_stamp))
                print(total_backloss/self.episode_len)
                print("Avg Cost per day")
                total_cost=total_overflow*self.sc+total_backloss*self.bc+sum(self.reg_order_history[self.time_stamp-self.episode_len:])*self.rc+sum(self.exp_order_history[self.time_stamp-self.episode_len:])*self.ec
                print(total_cost/self.episode_len)

# config={'regular_leadtime':6,'express_leadtime':2,'regular_cost':10,'express_cost':20,"max_order":20,"max_inventory":50,
#        'store_cost':4,'back_cost':35,'y':0.9,'starting_state':[20,[0 for _ in range(6)],[0 for _ in range(2)]]}
# game=DualSourcing(config,episode_len=20)
# game.play()



Matplotlib created a temporary config/cache directory at /var/folders/_x/fh3t8wcj3_xbs4fddcxmbpc00000gn/T/matplotlib-proejoj0 because the default path (/Users/hansshen/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
# ... [Your DualSourcing class code here] ...

class DualSourcingApp(tk.Tk):
    def __init__(self, game):
        super().__init__()
        self.game = game
        
        # Static labels for costs
        backorder_label = ttk.Label(self, text="Backorder Cost: " + str(game.bc), font=('Arial', 14, 'bold'))
        backorder_label.grid(row=0, column=0, padx=30,pady=10, sticky='w')

        storage_label = ttk.Label(self, text="Storage Cost: " + str(game.sc), font=('Arial', 14, 'bold'))
        storage_label.grid(row=1, column=0, padx=30,pady=10, sticky='w')

        express_label = ttk.Label(self, text="Express Cost: " + str(game.ec), font=('Arial', 14, 'bold'))
        express_label.grid(row=2, column=0, padx=30,pady=10, sticky='w')

        regular_label = ttk.Label(self, text="Regular Cost: " + str(game.rc), font=('Arial', 14, 'bold'))
        regular_label.grid(row=3, column=0, padx=30, pady=10, sticky='w')

        # Current demand label
        self.demand_label = ttk.Label(self, text="Current Demand: 0", font=('Verdana', 14, 'bold','underline'))
        self.demand_label.grid(row=4, column=0, pady=20, sticky='w')

        # Inventory label
        self.inventory_label = ttk.Label(self, text="Current Inventory: 0", font=('Verdana', 14, 'bold','underline'))
        self.inventory_label.grid(row=4, column=1, pady=20, sticky='w')

        # Pipelines label and Images
        self.pipeline_regular_label = ttk.Label(self, text="Pipeline (Regular): []", font=('Verdana', 14, 'bold'))
        self.pipeline_regular_label.grid(row=5, column=0, pady=10, sticky='w')
        
        self.truck_img = tk.PhotoImage(file="./truck_icon.png").subsample(6,6)
        truck_label = ttk.Label(self, image=self.truck_img)
        truck_label.grid(row=6, column=0, pady=10)

        self.pipeline_express_label = ttk.Label(self, text="Pipeline (Express): []", font=('Verdana', 14, 'bold'))
        self.pipeline_express_label.grid(row=7, column=0, pady=10, sticky='w')

        self.racecar_img = tk.PhotoImage(file="./racecar_icon.png").subsample(2, 2)
        racecar_label = ttk.Label(self, image=self.racecar_img)
        racecar_label.grid(row=8, column=0, pady=10)

        # Dynamic label for total costs
        self.total_costs_label = ttk.Label(self, text="Total Costs: 0",font=('Verdana', 16, 'bold','underline',),foreground="red")
        self.total_costs_label.grid(row=9, column=0, pady=20)

        # Entry boxes for orders
        self.reg_order_label= ttk.Label(self, text="Place Regular Order: ", font=('Arial', 14, 'bold'))
        self.reg_order_label.grid(row=10, column=0, pady=10)
        self.reg_order_entry = ttk.Entry(self)
        self.reg_order_entry.grid(row=10, column=1, pady=10)

        self.exp_order_label= ttk.Label(self, text="Place Express Order: ", font=('Arial', 14, 'bold'))
        self.exp_order_label.grid(row=11, column=0, pady=10)
        self.exp_order_entry = ttk.Entry(self)
        self.exp_order_entry.grid(row=11, column=1, pady=10)

        # Button to progress the game
        self.progress_button = ttk.Button(self, text="Make Order", command=self.make_order)
        self.progress_button.grid(row=12, column=0, pady=20)
        
        # Reserve Space for graphs
        self.demand_graph_frame = ttk.Frame(self)
        self.demand_graph_frame.grid(row=0, column=4, rowspan=6, columnspan=3, sticky="nsew")
        self.demand_graph_frame.grid_rowconfigure(0, weight=1)
        self.demand_graph_frame.grid_columnconfigure(0, weight=1)
        
        self.order_graph_frame = ttk.Frame(self)
        self.order_graph_frame.grid(row=6, column=4, rowspan=6, columnspan=3, sticky="nsew")
        self.order_graph_frame.grid_rowconfigure(0, weight=1)
        self.order_graph_frame.grid_columnconfigure(0, weight=1)

        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(4, weight=1)
        self.canvas_widget1,self.canvas_widget2=None,None

        # Refresh initial values
        self.refresh_labels()
    
        
        
    def embed_graph(self,fig1,fig2):
        if self.canvas_widget1:
            self.canvas_widget1.destroy()
            self.canvas_widget2.destroy()
        canvas1 = FigureCanvasTkAgg(fig1, master=self.demand_graph_frame)
        canvas2 = FigureCanvasTkAgg(fig2, master=self.order_graph_frame)
        self.canvas_widget1 = canvas1.get_tk_widget()
        self.canvas_widget1.grid(row=0, column=0, sticky="nsew")
        self.canvas_widget2 = canvas2.get_tk_widget()
        self.canvas_widget2.grid(row=0, column=0, sticky="nsew")
        canvas1.draw()
        canvas2.draw()
    
    def get_graph(self):
        fig1=plt.figure(figsize=[2,1])
        fig2=plt.figure(figsize=[2,1])
        ax_inv=fig1.add_subplot()
        ax_inv.set_xlabel("TimeStamp")
        ax_inv.set_ylabel("Inventory Level")
        ax_demand=fig2.add_subplot()
        ax_demand.set_xlabel("TimeStamp")
        ax_demand.set_ylabel("Demand Level")
#         ax_orders=fig.add_subplot(2,2,4)
#         ax_orders.set_xlabel("TimeStamp")
#         ax_orders.set_ylabel("Order Quantity")
        ax_inv.plot(list(range(self.game.time_stamp)),self.game.inventory_history)
        ax_demand.plot(list(range(self.game.time_stamp)),self.game.demand_history)
#         ax_orders.plot(list(range(self.time_stamp)),self.reg_order_history,label="Regular Orders")
#         ax_orders.plot(list(range(self.time_stamp)),self.exp_order_history,label="Express Orders")
#         ax_orders.legend()
        self.embed_graph(fig1,fig2)
        
    
    def make_order(self):
        reg_order = int(self.reg_order_entry.get())
        exp_order = int(self.exp_order_entry.get())
        self.game.step(reg_order, exp_order)
        self.refresh_labels()
        print(self.game.time_stamp)
        print(self.game.inventory_history)
        self.get_graph()

    def refresh_labels(self):
        self.demand_label["text"] = "Current Demand: " + str(self.game.demand[self.game.time_stamp])
        self.inventory_label["text"] = "Current Inventory: " + str(self.game.state[0])
        self.pipeline_regular_label["text"] = "Pipeline (Regular): " + str(self.game.state[1])
        self.pipeline_express_label["text"] = "Pipeline (Express): " + str(self.game.state[2])
        self.total_costs_label["text"] = "Total Costs: " + str(self.game.total_cost)  # Assuming you have a 'total_cost' attribute in your game

if __name__ == "__main__":
    config = {
        'regular_leadtime': 3,
        'express_leadtime': 1,
        'regular_cost': 1,
        'express_cost': 3,
        'store_cost': 0.5,
        'back_cost': 4,
        'max_order': 10,
        'max_inventory': 20,
        'y': 0.99,
        'starting_state': [10, [0, 0, 0], [0]]
    }

    game = DualSourcing(config)
    app = DualSourcingApp(game)
    app.mainloop()


1
[6]
2
[6, 4]
3
[6, 4, 1]
4
[6, 4, 1, -2]
5
[6, 4, 1, -2, -5]
6
[6, 4, 1, -2, -5, -6]
7
[6, 4, 1, -2, -5, -6, -6]
8
[6, 4, 1, -2, -5, -6, -6, -9]
9
[6, 4, 1, -2, -5, -6, -6, -9, -12]
10
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15]
11
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16]
12
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18]


/var/folders/_x/fh3t8wcj3_xbs4fddcxmbpc00000gn/T/ipykernel_45474/2002458475.py:97: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig1=plt.figure(figsize=[2,1])


13
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17]
14
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20]
15
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20]
16
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20]
17
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20]
18
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20]
19
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20]
20
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20, -18]
21
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20, -18, -20]
22
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20, -18, -20, -20]
23
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20, -18, -20, -20, -20]
24
[6, 4, 1, -2, -5, -6, -6, -9, -12, -15, -16, -18, -17, -20, -20, -20, -20, -20, -20,

In [ ]:
# old version 

class DualSourcingApp(tk.Tk):
    def __init__(self, game):
        super().__init__()
        self.game = game
        
        # Static labels for costs
        ttk.Label(self, text="Backorder Cost: " + str(game.bc),font=('Arial',14,'bold')).pack( pady=10)
        ttk.Label(self, text="Storage Cost: " + str(game.sc),font=('Arial', 14,'bold')).pack(pady=10)
        ttk.Label(self, text="Express Cost: " + str(game.ec),font=('Arial', 14, 'bold'), foreground="red").pack(pady=10)
        ttk.Label(self, text="Regular Cost: " + str(game.rc),font=('Arial', 14, 'bold'), foreground="blue").pack(pady=10)
        
        # Current demand label
        self.demand_label = ttk.Label(self, text="Current Demand: 0",font=('Verdana',14,'bold'))
        self.demand_label.pack(pady=20)
        
        # Inventory label
        self.inventory_label = ttk.Label(self, text="Current Inventory: 0",font=('Verdana',14,'bold'))
        self.inventory_label.pack(pady=20)
        
        # Pipelines label
        self.pipeline_regular_label = ttk.Label(self, text="Pipeline (Regular): []",font=('Verdana',14,'bold'))
        self.pipeline_regular_label.pack(pady=10)
        
        self.pipeline_express_label = ttk.Label(self, text="Pipeline (Express): []",font=('Verdana',14,'bold'))
        self.pipeline_express_label.pack(pady=10)
        
        #images 
        self.racecar_img = tk.PhotoImage(file="./racecar_icon.png")
        racecar_label = ttk.Label(self, image=self.racecar_img)
        racecar_label.pack(pady=10)
        
        
        # Dynamic label for total costs
        self.total_costs_label = ttk.Label(self, text="Total Costs: 0")
        self.total_costs_label.pack(pady=20)
        
        # Entry boxes for orders
        self.reg_order_entry = ttk.Entry(self)
        self.reg_order_entry.pack(pady=10)
        self.exp_order_entry = ttk.Entry(self)
        self.exp_order_entry.pack(pady=10)
        
        # Button to progress the game
        self.progress_button = ttk.Button(self, text="Make Order", command=self.make_order)
        self.progress_button.pack(pady=20)

        # Refresh initial values
        self.refresh_labels()

    def make_order(self):
        reg_order = int(self.reg_order_entry.get())
        exp_order = int(self.exp_order_entry.get())
        self.game.step(reg_order, exp_order)
        self.refresh_labels()
        self.game.plot(self.game.time_stamp - 1, self.game.time_stamp)

    def refresh_labels(self):
        self.demand_label["text"] = "Current Demand: " + str(self.game.demand[self.game.time_stamp])
        self.inventory_label["text"] = "Current Inventory: " + str(self.game.state[0])
        self.pipeline_regular_label["text"] = "Pipeline (Regular): " + str(self.game.state[1])
        self.pipeline_express_label["text"] = "Pipeline (Express): " + str(self.game.state[2])
        self.total_costs_label["text"] = "Total Costs: " + str(self.game.total_cost)  # Assuming you have a 'total_cost' attribute in your game

if __name__ == "__main__":
    config = {
        'regular_leadtime': 3,
        'express_leadtime': 1,
        'regular_cost': 1,
        'express_cost': 3,
        'store_cost': 0.5,
        'back_cost': 0.5,
        'max_order': 10,
        'max_inventory': 20,
        'y': 0.99,
        'starting_state': [10, [0, 0, 0], [0]]
    }

    game = DualSourcing(config)
    app = DualSourcingApp(game)
    app.mainloop()
